In [1]:
# Code for ETL operations on Country-GDP data
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import logging
import sqlite3
# Importing the required libraries

url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'

def log_progress(message):
    ''' This function logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing'''
    with open('code_log.txt','a') as log_file:
        log_file.write(message + '\n')

log_progress('Preliminaries complete. Initiating ETL process')

def extract(url, table_attribs):
    ''' This function aims to extract the required
    information from the website and save it to a data frame. The
    function returns the data frame for further processing. '''
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table') #adjust this to math the table's HTML tag

        rows = []
        for tr in table.find_all('tr')[1:]: #skip the header row
            cells = []
            for td in tr.find_all('td'):
                cells.append(td.text.strip())
            rows.append(cells)
        
        df = pd.DataFrame(rows, columns = table_attribs )
        log_progress('Data extraction complete. Initiating Transformation process')
        return df
    else:
        return None  

table_attribs = ['rank','Name', 'MC_USD_Billion']
df = extract(url,table_attribs)
print(df)
print(df['MC_USD_Billion'].apply(type))
df['MC_USD_Billion'] = pd.to_numeric(df['MC_USD_Billion'], errors='coerce')
print(df['MC_USD_Billion'].apply(type))


def transform(df, csv_path):
    ''' This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies'''

    # Leer el archivo CSV de tasas de cambio y convertirlo en un diccionario
    exchange_rate_df = pd.read_csv(csv_path)
    exchange_rate = exchange_rate_df.set_index('Currency').to_dict()['Rate']

    df1 = pd.read_csv(csv_path)
    df['MC_GBP_Billion'] = [np.round(x*exchange_rate['GBP'],2) for x in df['MC_USD_Billion']]
    df['MC_EUR_Billion'] = [np.round(x*exchange_rate['EUR'],2) for x in df['MC_USD_Billion']]
    df['MC_INR_Billion'] = [np.round(x*exchange_rate['INR'],2) for x in df['MC_USD_Billion']]

    log_progress('Data transformation complete. Initiating Loading process')
    return df

df_transformed = transform(df,'exchange_rate.csv')
print(df_transformed)

def load_to_csv(df, output_path):
    ''' This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing.'''
    df.to_csv(output_path, index = False)
    log_progress('Data saved to CSV file')

load_to_csv(df_transformed,'transformed_file.csv')

def load_to_db(df, sql_connection, table_name):
    ''' This function saves the final data frame to a database
    table with the provided name. Function returns nothing.'''
    df.to_sql(table_name,sql_connection, if_exists='replace', index=False)
    log_progress('Data loaded to Database as a table, Executing queries')

sql_connection = sqlite3.connect('database.db')
load_to_db(df_transformed, sql_connection, 'largest_banks')


def run_query(query_statement, sql_connection):
    ''' This function runs the query on the database table and
    prints the output on the terminal. Function returns nothing. '''
    cursor = sql_connection.cursor()
    print(f"Executing query: {query_statement}")
    cursor.execute(query_statement)
    results = cursor.fetchall()
    for row in results:
        print(row)
    log_progress('Process Complete')

query1 = "SELECT * FROM Largest_banks"
query2 = "SELECT AVG(MC_GBP_Billion) FROM Largest_banks"
query3 = "SELECT Name from Largest_banks LIMIT 5"

run_query(query1,sql_connection)
run_query(query2,sql_connection)
run_query(query3,sql_connection)

sql_connection.close()
log_progress('Server Connection closed')

''' Here, you define the required entities and call the relevant
functions in the correct order to complete the project. Note that this
portion is not inside any function.'''

  rank                                     Name MC_USD_Billion
0    1                           JPMorgan Chase         432.92
1    2                          Bank of America         231.52
2    3  Industrial and Commercial Bank of China         194.56
3    4               Agricultural Bank of China         160.68
4    5                                HDFC Bank         157.91
5    6                              Wells Fargo         155.87
6    7                        HSBC Holdings PLC         148.90
7    8                           Morgan Stanley         140.83
8    9                  China Construction Bank         139.82
9   10                            Bank of China         136.81
0    <class 'str'>
1    <class 'str'>
2    <class 'str'>
3    <class 'str'>
4    <class 'str'>
5    <class 'str'>
6    <class 'str'>
7    <class 'str'>
8    <class 'str'>
9    <class 'str'>
Name: MC_USD_Billion, dtype: object
0    <class 'float'>
1    <class 'float'>
2    <class 'float'>
3    <class 'float

FileNotFoundError: [Errno 2] No such file or directory: 'exchange_rate.csv'